## Классификация текстов с использованием предобученных языковых моделей.

В данном задании вам предстоит обратиться к задаче классификации текстов и решить ее с использованием предобученной модели BERT.

In [1]:
import json
# do not change the code in the block below
# __________start of block__________
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

%matplotlib inline
# __________end of block__________

Обратимся к набору данных SST-2. Holdout часть данных (которая понадобится вам для посылки) доступна по ссылке ниже.

In [2]:
# do not change the code in the block below
# __________start of block__________

!wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
# __________end of block__________

--2025-04-04 06:49:54--  https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51581 (50K) [text/plain]
Saving to: ‘texts_holdout.json’

texts_holdout.json  100%[===================>]  50.37K  --.-KB/s    in 0.02s   

2025-04-04 06:49:54 (2.94 MB/s) - ‘texts_holdout.json’ saved [51581/51581]



In [3]:
# do not change the code in the block below
# __________start of block__________
df = pd.read_csv(
    "https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",
    delimiter="\t",
    header=None,
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]
with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)
# __________end of block__________

In [ ]:
import numpy as np
print(np.unique(y_train, return_counts=True))

(array([0, 1]), array([2393, 2607]))


In [ ]:
df.iloc[124][1]

np.int64(1)

In [ ]:
set(y_train)

{np.int64(0), np.int64(1)}

Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

Сначал разобьем текст на токены для берта. Это делается встроенным токенатором

In [27]:
from transformers import BertTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import DataCollatorWithPadding

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,          # Включить маскирование
    mlm_probability=0.15  # Вероятность маскирования (15% по умолчанию)
)

  В берте когда мы обучаем его на понимании речи с помощью mask токенов labels - указывает токены типа MASK для подчета лосс. обычно такие токены это 103, в labels на их метсах стоят реальные токены а на остальных -100 чтобы не участовать в лоссе. Это нужно указать в datacollar. Модель для токена маски выдает вектор скрытого представления и из него строит вероятностное распределения слов в словаре и предсказывает наиболее вероятное слово.

  Как будто бы в нашей задаче не нужно дообучать берт на таком подходе тк он уже обучен на массивах текста в разы больше нашего.

  **Сначала просто попробуем обучить классификатор и соответственно дообучить сам берт на распознавании оуенки пользователя. **

In [28]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt' # возвращает тензор размерностью (1,batch_size)
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(0),  # Убираем batch-размерность [1, max_len] → [max_len]
            "attention_mask": inputs["attention_mask"].squeeze(0),
        }

НЕ НАДО СМЕШИВАТЬ ЗАДАЧИ

классификация и обучение берта на понимаии речи это разные задачи.

когда решаем вторую то нужен еще колоратор чтобы сделать токены маскированными (в даталоудере указываем collate_fn=collator)

Ну и для второй задачи надо использовать немнорго другубю модель специальнро для mlm так как там для каждого mask токена скрвтое его состаяние передается в классифкатор и предсказыввается слово. Довольнро сложно реализовать на базовой архитектуре


In [29]:
from torch.utils.data import DataLoader

tokinize_text_train = CustomDataset(texts_train, y_train, tokenizer, max_len=40 )
tokinize_text_test = CustomDataset(texts_test, y_test, tokenizer, max_len=40 )

train_dl = DataLoader(tokinize_text_train, batch_size=128, '''collate_fn=data_collator''', shuffle=True) # пример для обучении на задаче mlm c collator
test_dl = DataLoader(tokinize_text_test, batch_size=128, '''collate_fn=data_collator''', shuffle=False)

In [30]:
tokinize_text_train[0]['input_ids']

tensor([  101,  1037, 18385,  1010,  6057,  1998,  2633, 18276,  2128, 16603,
         1997,  5053,  1998,  1996,  6841,  1998,  5687,  5469,  3152,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [33]:
batch = [tokinize_text_train[i] for i in range(2)]  # Берем 2 примера
collated = data_collator(batch)
print(collated['input_ids'].shape)

torch.Size([2, 40])


In [32]:
tokinize_text_train[125]

{'input_ids': tensor([  101,  2720, 15616,  2003,  1010,  2004,  4038,  3632,  1010,  2200,
         10021,  1998,  1999,  1996,  2190,  2126,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
for batch, label in train_dl:
  print(batch['input_ids'].size())
  break

torch.Size([128, 40])


Вообще есть уже встроенный класс для классифкации в берт. Однако работает он буквально как мой код, поэтому ради практики напишу свой

In [ ]:
class BertClassifier1(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert

        # Усиленная классификационная головка
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),  # Увеличиваем dropout
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),  # Добавляем BatchNorm
            nn.Dropout(0.2),     # Дополнительный dropout
            nn.Linear(256, 1)
        )

        # Правильная инициализация весов
        self._init_weights()

    def _init_weights(self):
        for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True  # Получаем все скрытые состояния
        )

        # Вариант 1: Используем pooler_output
        pooled_output = outputs.pooler_output

        return self.classifier(pooled_output)

In [ ]:
tokinize_text_train[0][0]

{'input_ids': tensor([  101,  1037, 18385,  1010,  6057,  1998,  2633, 18276,  2128, 16603,
          1997,  5053,  1998,  1996,  6841,  1998,  5687,  5469,  3152,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
print(f"Размер словаря токенизатора: {tokenizer.vocab_size}")
print(f"Размер embedding-слоя модели: {model.bert.embeddings.word_embeddings.num_embeddings}")

Размер словаря токенизатора: 30522
Размер embedding-слоя модели: 30522


In [ ]:
model = BertClassifier1(bert)
device = 'cuda'
model.to(device)
for i, d in train_dl:

  input_ids = i['input_ids'].to(device)
  attention_mask = i['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  print(output.shape)
  print(input_ids.shape)
  print(d.shape)
  break

torch.Size([128, 1])
torch.Size([128, 40])
torch.Size([128])


Обучение

In [ ]:
from tqdm import tqdm
import torch
def train(model, optimizer, criterion, train_dl, test_dl):
  model.train()
  for epoch in range(10):
    train_losses = 0
    norm = 0
    accuracy = 0
    model.to(device)
    total_acc = 0
    for data, label in tqdm(train_dl, desc="Training"):
      optimizer.zero_grad()

      input_ids = data['input_ids'].to(device)
      attention_mask = data['attention_mask'].to(device)

      label = label.float().unsqueeze(1).to(device)
      output = model(input_ids, attention_mask)
      loss = criterion(output, label)
      loss.backward()
      train_losses += loss.item()*input_ids.size(0)
      norm += input_ids.size(0)

      preds = (output >= 0.5).float()
      acc = (preds == label).float().mean()
      total_acc += acc.item()

      optimizer.step()

    print(f"Train loss: {train_losses/norm}", total_acc/norm)

    model.eval()
    test_losses = 0
    norm = 0
    with torch.no_grad():
      for data, label in tqdm(test_dl, desc="Testing"):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)

        label = label.float().unsqueeze(1).to(device)

        output = model(input_ids, attention_mask)
        loss = criterion(output, label)
        test_losses += loss.item()*input_ids.size(0)
        norm += input_ids.size(0)

    print(f"Test loss: {test_losses/norm}")





Полное обучение модели сол всеми размороженными весами дает плохой результат и забывание. Заморозим Берт

In [ ]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-uncased')
model = BertClassifier1(bert)


for params in model.bert.parameters():
  params.requires_grad = False

for name, param in model.named_parameters():
    print(f"Параметр {name} обучаемый: {param.requires_grad}")

Параметр bert.embeddings.word_embeddings.weight обучаемый: False
Параметр bert.embeddings.position_embeddings.weight обучаемый: False
Параметр bert.embeddings.token_type_embeddings.weight обучаемый: False
Параметр bert.embeddings.LayerNorm.weight обучаемый: False
Параметр bert.embeddings.LayerNorm.bias обучаемый: False
Параметр bert.encoder.layer.0.attention.self.query.weight обучаемый: False
Параметр bert.encoder.layer.0.attention.self.query.bias обучаемый: False
Параметр bert.encoder.layer.0.attention.self.key.weight обучаемый: False
Параметр bert.encoder.layer.0.attention.self.key.bias обучаемый: False
Параметр bert.encoder.layer.0.attention.self.value.weight обучаемый: False
Параметр bert.encoder.layer.0.attention.self.value.bias обучаемый: False
Параметр bert.encoder.layer.0.attention.output.dense.weight обучаемый: False
Параметр bert.encoder.layer.0.attention.output.dense.bias обучаемый: False
Параметр bert.encoder.layer.0.attention.output.LayerNorm.weight обучаемый: False
Параме

In [ ]:
device = 'cuda'
model.to(device)

BertClassifier1(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
print("Пример текста:", texts_train[0])
print("Токенизированный:", tokenizer.tokenize(texts_train[0]))

Пример текста: a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
Токенизированный: ['a', 'stirring', ',', 'funny', 'and', 'finally', 'transporting', 're', 'imagining', 'of', 'beauty', 'and', 'the', 'beast', 'and', '1930s', 'horror', 'films']


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
criterion = nn.BCEWithLogitsLoss()
train(model, optimizer, criterion, train_dl, test_dl)

Training: 100%|██████████| 40/40 [00:11<00:00,  3.43it/s]


Train loss: 0.9080865430831909 0.00395625


Testing: 100%|██████████| 15/15 [00:04<00:00,  3.28it/s]


Test loss: 0.6855356335639954


Training: 100%|██████████| 40/40 [00:11<00:00,  3.53it/s]


...

Test loss: 0.6030762632687886


Training: 100%|██████████| 40/40 [00:11<00:00,  3.44it/s]


Train loss: 0.5963711725234986 0.0052546875


Testing: 100%|██████████| 15/15 [00:04<00:00,  3.02it/s]

Test loss: 0.5963854193687439


Дробавим классифкатор на выход модели

#### Сдача взадания в контест
Сохраните в словарь `out_dict` вероятности принадлежности к первому (положительному) классу

In [ ]:
out_dict = {
    'train': # list of length 5000 with probas
    'test': # list of length 1920 with probas
    'holdout': # list of length 500 with probas
}

Несколько `assert`'ов для проверки вашей посылки:

In [ ]:
assert isinstance(out_dict["train"], list), "Object must be a list of floats"
assert isinstance(out_dict["train"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["train"]) == 5000
), "The predicted probas list length does not match the train set size"

assert isinstance(out_dict["test"], list), "Object must be a list of floats"
assert isinstance(out_dict["test"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["test"]) == 1920
), "The predicted probas list length does not match the test set size"

assert isinstance(out_dict["holdout"], list), "Object must be a list of floats"
assert isinstance(out_dict["holdout"][0], float), "Object must be a list of floats"
assert len(
    out_dict["holdout"] == 500
), "The predicted probas list length does not match the holdout set size"

Запустите код ниже для генерации посылки.

In [ ]:
# do not change the code in the block below
# __________start of block__________
FILENAME = "submission_dict_hw_text_classification_with_bert.json"

with open(FILENAME, "w") as iofile:
    json.dump(out_dict, iofile)
print(f"File saved to `{FILENAME}`")
# __________end of block__________

На этом задание завершено. Поздравляем!